In [ ]:
import json
posts = json.load(open('posts.json'))
len(posts), posts[0]

In [ ]:
text = '\n'.join(posts).lower()
len(text)

In [ ]:
characters = sorted(list(set(text)))
char_lookup = characters.index
len(characters)

In [ ]:
from tqdm import tqdm

maxlen = 100
X, Y = [], []
for i in tqdm(range(len(text) - maxlen)):
    x = text[i:i+maxlen]
    y = text[i+maxlen]
    X.append([char_lookup(c) for c in x])
    Y.append(char_lookup(y))
len(X), len(Y)

In [ ]:
import numpy as np
import keras.utils

X = np.reshape(X, (len(X), maxlen)) / len(characters) # normalized seq input
Y = keras.utils.to_categorical(Y)

X.shape, Y.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Flatten, Dropout

In [ ]:
model = Sequential()
model.add(Embedding(len(characters), 40, input_length=maxlen, name="Embedding"))
model.add(LSTM(256, name="LSTM"))
model.add(Dropout(0.2, name="Dropout"))
model.add(Dense(len(characters), activation='softmax', name="FullyConnected"))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, Y, epochs=1, batch_size=256, shuffle=True)

In [ ]:
char_reverse_lookup = characters.__getitem__
char_reverse_lookup(0)

In [ ]:
def __make_sentence__(length=420):
    seq = X[np.random.randint(0, len(X))].copy().tolist()
    ret = ''.join([char_reverse_lookup(int(i)) for i in seq])
    for i in tqdm(range(length)):
        xhat = np.reshape(seq, (-1, 100)) / len(seq)
        yhat = model.predict(xhat)
        result = char_reverse_lookup(np.argmax(yhat))
        ret += result
        seq = seq[1:]
        seq.append(np.argmax(yhat))
    return ret

model.make_sentence = __make_sentence__

model.make_sentence()

In [ ]:
model.fit(X, Y, epochs=50, batch_size=256, shuffle=True)